# Dynamic Router

Bridgic can dynamic routing to the corresponding worker based on the runtime conditions. Now let's understand them with a sample example.

<br>
<div style="text-align: center;">
<img src="../imgs/dynamic_router.png" alt="Parameter Passing" width="400" height="250">
</div>
<br>

Dynamic routing is a common requirement in chatbots. For instance, it involves analyzing the input query to determine the main domain or field it pertains to, and then dynamically routing it to the corresponding processing logic.

The user inputs the original query, we identify whether it is a medical-related or legal-related issue, and then we answer it by specialized logic. There are three steps:

1. Receive user input
2. Domain recognition and dynamic routing to corroct worker
3. Answer query

Before we start, let's prepare the running environment.

Use the `export` command to set up the API information for model invocation.

In [ ]:
# Setting environment variables in the terminal:
export VLLM_SERVER_API_BASE="your-api-url"
export VLLM_SERVER_API_KEY="your-api-key"
export VLLM_SERVER_MODEL_NAME="your-model-name"

Get the environment variables.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

Import the necessary packages.

In [ ]:
from typing import List, Dict, Tuple
from bridgic.core.automa import GraphAutoma, worker
from bridgic.llms.vllm.vllm_server_llm import VllmServerLlm, Message, Role, Choice

We assume that the user query we receive is a string, let's implement this dynamic router to answer user query.

We initialize the model to facilitate our subsequent convenient invocation of it to complete tasks.

In [ ]:
llm = VllmServerLlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=5,
)

Next, let's complete the two steps of Dynamic router to achieve our goal:

In [ ]:
class DomainChatbot(GraphAutoma):
    @worker(is_start=True)
    async def pre_query(self, query: str):  # receive user input and preprocess it
        return query

    @worker(dependencies=["pre_query"])
    async def domain_recognition(self, query: str):  # domain recognition and dynamic routing
        response: str = await llm.astructured_output(
            model=_model_name,
            constraint=Choice(choices=["medical", "legal"]),
            messages=[
                Message.from_text(text="Please identify the domain of the following query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        
        # dynamic routing
        if response == "medical":
            return self.ferry_to('answer_medical_query', query)
        elif response == "legal":
            return self.ferry_to('answer_legal_query', query)
    
    @worker(is_output=True)
    async def answer_medical_query(self, query: str):  # answer medical-related query
        response: str = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a medical expert. Answer the following medical-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response

    @worker(is_output=True)
    async def answer_legal_query(self, query: str):  # answer legal-related query
        response: str = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a legal expert. Answer the following legal-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response

    

Let's run it!

In [ ]:
query = "What is the asptite?"
domain_chatbot = DomainChatbot(output_worker_key="answer_medical_query")
await domain_chatbot.run(query)